#### Mixed model + variance parameter estimation

Consider the model:

$\mathbf{y} = \mathbf{X\beta+Za+\epsilon}$, where

Consider the vector $\mathbf{y}$ - a vector with phenotypic values ($n \times 1$ in our case where $n$ is the number of animals).

vector $\mathbf{\beta}$ represents the vector of fixed effects ($p \times 1$), where $p$ is the number of fixed effects.

vector $\mathbf{a}$ represents the vector of random effects ($q \times 1$), where $q$ is the number of random effects.

vector $\mathbf{\epsilon}$ represents the vector of errors or residuals ($n \times 1$), where $n$ is the number of observations or data points.

The matrix $\mathbf{X}$ is the design matrix for fixed effects, with dimensions $n \times p$, where $n$ is the number of observations or data points and $p$ is the number of fixed effects.

matrix $\mathbf{Z}$ is the design matrix for random effects, with dimensions $n \times q$, where $n$ is the number of observations or data points, and $q$ is the number of random effects.

We assume that the matrices $\mathbf{X}$ and $\mathbf{Z}$ are independent. Additionally, we assume that:


$E(\mathbf{y}) = \mathbf{X\beta}$, $E(\mathbf{a})=E(\mathbf{\epsilon})=0$

$V=Var(\mathbf{y})=Var(\mathbf{Za})+ Var(\mathbf{\epsilon})=\mathbf{Z}Var(\mathbf{a})\mathbf{Z}^{T}+\mathbf{I}Var(\mathbf{\epsilon})\mathbf{I}^{T}=\mathbf{ZGZ}^{T}+\mathbf{R}$, where

$\mathbf{G}=\mathbf{A}\cdot\sigma^{2}_{a}$, $\mathbf{R}=\mathbf{I}\cdot\sigma^{2}_{\epsilon}$

$ \left[ \begin{array}{cc}
         \mathbf{X}^{T}\mathbf{X} & \mathbf{X}^{T}\mathbf{Z} \\
         \mathbf{Z}^{T}\mathbf{X} & \mathbf{Z}^{T}\mathbf{Z}+\mathbf{A}^{-1}\alpha \end{array}\right]\cdot
  \left[ \begin{array}{c}
         \widehat{\mathbf{\beta}} \\
         \widehat{\mathbf{a}} \end{array}\right]=
  \left[ \begin{array}{c}
         \mathbf{X}^{T}\mathbf{y} \\
         \mathbf{Z}^{T}\mathbf{y} \end{array}\right]$, where

$\alpha=\frac{\sigma^{2}_{\epsilon}}{\sigma^{2}_{a}}$

$ \left[ \begin{array}{c}
         \widehat{\mathbf{\beta}} \\
         \widehat{\mathbf{a}} \end{array}\right]=
  \left[ \begin{array}{cc}
         \mathbf{X}^{T}\mathbf{X} & \mathbf{X}^{T}\mathbf{Z} \\
         \mathbf{Z}^{T}\mathbf{X} & \mathbf{Z}^{T}\mathbf{Z}+\mathbf{A}^{-1}\alpha \end{array}\right]^{-1}\cdot
  \left[ \begin{array}{c}
         \mathbf{X}^{T}\mathbf{y} \\
         \mathbf{Z}^{T}\mathbf{y} \end{array}\right]$

$ C = \left[ \begin{array}{cc}
         \mathbf{X}^{T}\mathbf{X} & \mathbf{X}^{T}\mathbf{Z} \\
         \mathbf{Z}^{T}\mathbf{X} & \mathbf{Z}^{T}\mathbf{Z}+\mathbf{A}^{-1}\alpha \end{array}\right] = 
         \left[ \begin{array}{cc}
         \mathbf{C}_{11} & \mathbf{C}_{12} \\ \mathbf{C}_{21} & \mathbf{C}_{22} \end{array}\right]$

$ C^{-1} = \left[ \begin{array}{cc}
         \mathbf{C}^{11} & \mathbf{C}^{12} \\ \mathbf{C}^{21} & \mathbf{C}^{22} \end{array}\right]$

The precision of the estimation is given by $r^2 = \text{diag}(1 - \mathbf{C}^{22} \cdot \alpha)$, where $\mathbf{C}^{22}$ is a matrix, and $\alpha$ is a parameter or another matrix involved in the calculation.

1. MATRIX A

In [ ]:
library(AGHmatrix)

id = 1:8
sire = c(0, 0, 0, 1, 3, 1, 4, 3)
dam = c(0, 0, 0, 0, 2, 2, 5, 6)

ped = cbind(id, sire, dam)

In [ ]:
A = Amatrix(ped, ploidy = 2)

In [ ]:
(A = as.matrix(A))

2. We define the successive sources of information.

In [ ]:
y = as.matrix(c(4.5, 2.9, 3.9, 3.5, 5.0))
t(y)

sex = c(1, 0, 0, 1, 1)
X = matrix(0, 5, 2)
X[,1] = sex
X[,2] = 1-sex
X

model.matrix(~factor(sex) - 1)

In [ ]:
I = diag(5)
Z = matrix(0, 5, 8)
Z[1:5, 4:8] = I
Z

3. We solve the system of mixed equations.

In [ ]:
library(MASS)

mme = function(y, X, Z, A, sigma_a, sigma_e) {
    alpha = sigma_e / sigma_a
    invA = ginv(A)
    C = rbind(cbind(t(X)%*%X, t(X)%*%Z),
              cbind(t(Z)%*%X, t(Z)%*%Z+invA*c(alpha)))
    rhs = rbind(t(X)%*%y, t(Z)%*%y)
    invC = ginv(C)
    estimators = invC%*%rhs
    list(C = C, est = estimators)
}

mme(y, X, Z, A, 20, 40)

In [10]:
C = as.matrix(mme(y, X, Z, A, 20, 40)$C)
(invC = ginv(C))

invC22 = invC[3:10, 3:10]
(r2 = diag(1 - invC22*2))
(r = sqrt(r2))

0.59556097,0.15730213,-0.164119413,-0.083624565,-0.13059083,-0.26455749,-0.14827804,-0.16632621,-0.2842464,-0.2378790
0.15730213,0.80245865,-0.132863260,-0.241250738,-0.11196840,-0.08730803,-0.29891465,-0.30600266,-0.1859495,-0.1986488
-0.16411941,-0.13286326,0.471094211,0.006928037,0.03264668,0.21954371,0.04495225,0.22077427,0.1386223,0.1341923
-0.08362457,-0.24125074,0.006928037,0.492095721,-0.01030797,0.02039033,0.23734577,0.24515571,0.1198194,0.1106640
-0.13059083,-0.11196840,0.032646682,-0.010307967,0.45645878,0.04812709,0.20132326,0.02261354,0.1258983,0.2177471
-0.26455749,-0.08730803,0.219543709,0.020390333,0.04812709,0.42768015,0.04704420,0.12757186,0.2428012,0.1231911
-0.14827804,-0.29891465,0.044952254,0.237345770,0.20132326,0.04704420,0.42810675,0.16972255,0.2197160,0.1780739
-0.16632621,-0.30600266,0.220774267,0.245155707,0.02261354,0.12757186,0.16972255,0.44228277,0.1521830,0.2192238
-0.28424641,-0.18594950,0.138622268,0.119819354,0.12589831,0.24280124,0.21971599,0.15218301,0.4418562,0.1680818
-0.23787901,-0.19864885,0.134192262,0.110664009,0.21774710,0.12319108,0.17807393,0.21922376,0.1680818,0.4223641


[1] 0.05781158 0.01580856 0.08708243 0.14463969 0.14378651 0.11543447 0.11628766
[8] 0.15527171

[1] 0.2404404 0.1257321 0.2950973 0.3803153 0.3791919 0.3397565 0.3410098
[8] 0.3940453

4. Estimation of variance parameters.

In [11]:
var(y)

0.678


In [15]:
sigma_a = 1.01  #starting value for random effect
sigma_e = 10.01 #starting value for error variance

In [13]:
EM = function(y, X, Z, A, sigma_a, sigma_e) {
  n = nrow(X)
  p = ncol(X) 
  q = nrow(A) 
  
  t = 1 #iteration number 1
  tmp = 0.1 #test for convergance
  
  while (tmp > 0.00001) {
    mme_new = mme(y, X, Z, A, sigma_a, sigma_e)
    C_new = ginv(mme_new$C)
    Ck = C_new[(p+1):(p+q), (p+1):(p+q)]
    mme2 = mme_new$est
    
    a = as.matrix(mme2[(p+1):(p+q)])
    sigma_a_new = (t(a)%*%ginv(A)%*%a + sum(diag(ginv(A)%*%Ck))*c(sigma_e))/q
    
    res = as.matrix(y-X%*%as.matrix(mme2[1:p]) - Z%*%as.matrix(mme2[(p+1):(p+q)]))
    X.tmp1 = cbind(X,Z) %*% C_new
    X.tmp2 = t(cbind(X,Z))
    sigma_e_new = (t(res)%*%res + sum(diag(X.tmp1%*%X.tmp2))*c(sigma_e))/n
    
    tmp = max(abs(sigma_a - sigma_a_new), abs(sigma_e - sigma_e_new))
    sigma_a = sigma_a_new
    sigma_e = sigma_e_new
    
    t = t + 1
  }
  list(t = t, sigma_a = sigma_a, sigma_e = sigma_e)
}

$y=X\beta+Za+\epsilon$

$\epsilon = y - X\beta - Za$

#### $\sigma^{2}_{\epsilon[t+1]} = \frac{\widehat{\epsilon}^{'}_{[t]}\widehat{\epsilon}_{[t]} + tr([X, Z]C^{22}_{[t]}[X, Z]^{'})\cdot\sigma^{2}_{\epsilon[t]}}{n}$

#### $\sigma^{2}_{a[t+1]} = \frac{\widehat{a}^{'}_{[t]}A^{-1}\widehat{a}_{[t]} + tr(A^{-1}C^{22}_{[t]})\cdot\sigma^{2}_{\epsilon[t]}}{q}$

In [17]:
(wyniki = EM(y, X, Z, A, sigma_a, sigma_e))
wyniki$sigma_a+wyniki$sigma_e
var(y)

0.6555048
0.0180315


0.6735363


0.678


5. Heritability.

In [18]:
(h2 = 0.6555048 / (0.6555048 + 0.0180315))

[1] 0.9732286

In [23]:
cbind(mme(y, X, Z, A, wyniki$sigma_a, wyniki$sigma_e)$est,
      mme(y, X, Z, A, 20, 40)$est)[3:10,]

cor(cbind(mme(y, X, Z, A, wyniki$sigma_a, wyniki$sigma_e)$est,
      mme(y, X, Z, A, 20, 40)$est)[3:10,])

0.27628267,0.098444576
-0.12980570,-0.018770099
-0.08638891,-0.041084203
0.04800924,-0.008663123
-0.57427373,-0.185732099
0.40960922,0.176872088
-0.90253116,-0.249458555
0.54139768,0.182614688


1.0000000,0.9879948
0.9879948,1.0000000


6. Wald test with new $\sigma^{2}_{a}$ i $\sigma^{2}_{\epsilon}$

6.1 old variance parameters

In [27]:
G = A*c(20)
R = diag(5)*c(40)
V = Z%*%G%*%t(Z) + R
V

(varB = ginv(t(X)%*%ginv(V)%*%X))
(seB = sqrt(diag(varB)))

(testWalda = mme(y, X, Z, A, 20, 40)$est[1:2] / seB)
(p_value = round(2*pnorm(abs(testWalda), lower.tail = FALSE), digits = 10))

60.0,0.0,5,10,2.5
0.0,60.0,5,10,7.5
5.0,5.0,60,5,10.0
10.0,10.0,5,60,5.0
2.5,7.5,10,5,60.0


23.822439,6.292085
6.292085,32.098346


[1] 4.880824 5.665540

[1] 0.8929850 0.6009012

[1] 0.3718652 0.5479058

6.2 new variance parameters

In [26]:
G = A*c(wyniki$sigma_a)
R = diag(5)*c(wyniki$sigma_e)
V = Z%*%G%*%t(Z) + R
V

(varB = ginv(t(X)%*%ginv(V)%*%X))
(seB = sqrt(diag(varB)))

(testWalda = mme(y, X, Z, A, wyniki$sigma_a, wyniki$sigma_e)$est[1:2] / seB)
(p_value = round(2*pnorm(abs(testWalda), lower.tail = FALSE), digits = 10))

0.6735363,0.0000000,0.1638762,0.3277524,0.0819381
0.0000000,0.6735363,0.1638762,0.3277524,0.2458143
0.1638762,0.1638762,0.6735363,0.1638762,0.3277524
0.3277524,0.3277524,0.1638762,0.6735363,0.1638762
0.0819381,0.2458143,0.3277524,0.1638762,0.6735363


0.3424174,0.2026791
0.2026791,0.3633763


[1] 0.5851644 0.6028070

[1] 7.583695 5.776861

[1] 0.0e+00 7.6e-09

7. Assessment accuracy

7.1 old variance parameters

In [45]:
C = as.matrix(mme(y, X, Z, A, 20, 40)$C)
(invC = ginv(C))

invC22 = invC[3:10, 3:10]
(r2 = diag(1 - invC22*2))
(r = sqrt(r2))

0.59556097,0.15730213,-0.164119413,-0.083624565,-0.13059083,-0.26455749,-0.14827804,-0.16632621,-0.2842464,-0.2378790
0.15730213,0.80245865,-0.132863260,-0.241250738,-0.11196840,-0.08730803,-0.29891465,-0.30600266,-0.1859495,-0.1986488
-0.16411941,-0.13286326,0.471094211,0.006928037,0.03264668,0.21954371,0.04495225,0.22077427,0.1386223,0.1341923
-0.08362457,-0.24125074,0.006928037,0.492095721,-0.01030797,0.02039033,0.23734577,0.24515571,0.1198194,0.1106640
-0.13059083,-0.11196840,0.032646682,-0.010307967,0.45645878,0.04812709,0.20132326,0.02261354,0.1258983,0.2177471
-0.26455749,-0.08730803,0.219543709,0.020390333,0.04812709,0.42768015,0.04704420,0.12757186,0.2428012,0.1231911
-0.14827804,-0.29891465,0.044952254,0.237345770,0.20132326,0.04704420,0.42810675,0.16972255,0.2197160,0.1780739
-0.16632621,-0.30600266,0.220774267,0.245155707,0.02261354,0.12757186,0.16972255,0.44228277,0.1521830,0.2192238
-0.28424641,-0.18594950,0.138622268,0.119819354,0.12589831,0.24280124,0.21971599,0.15218301,0.4418562,0.1680818
-0.23787901,-0.19864885,0.134192262,0.110664009,0.21774710,0.12319108,0.17807393,0.21922376,0.1680818,0.4223641


[1] 0.05781158 0.01580856 0.08708243 0.14463969 0.14378651 0.11543447 0.11628766
[8] 0.15527171

[1] 0.2404404 0.1257321 0.2950973 0.3803153 0.3791919 0.3397565 0.3410098
[8] 0.3940453

7.2 new variance parameters

In [46]:
C = as.matrix(mme(y, X, Z, A, wyniki$sigma_a, wyniki$sigma_e)$C)
(invC = ginv(C))

invC22 = invC[3:10, 3:10]
alpha = wyniki$sigma_e / wyniki$sigma_a
(r2_new = diag(1 - invC22*c(alpha)))
(r_new = sqrt(r2_new))

18.989954,11.240278,-11.367991,-5.744381,-10.615597,-18.622016,-11.181726,-11.298831,-18.776774,-18.571073
11.240278,20.152307,-10.550520,-15.341541,-6.692544,-10.928327,-19.576775,-19.727839,-11.449505,-11.343003
-11.367991,-10.550520,29.600734,1.216585,7.735006,11.501854,10.245954,10.855086,11.342226,11.259894
-5.744381,-15.341541,1.216585,33.648374,-1.737283,5.446753,15.259658,15.423424,5.984717,5.801674
-10.615597,-6.692544,7.735006,-1.737283,25.984301,10.424441,7.022512,6.362575,10.516929,10.905420
-18.622016,-10.928327,11.501854,5.446753,10.424441,19.221329,10.860765,10.995889,18.435280,18.209439
-11.181726,-19.576775,10.245954,15.259658,7.022512,10.860765,19.979981,19.173568,11.402758,11.281656
-11.298831,-19.727839,10.855086,15.423424,6.362575,10.995889,19.173568,20.282111,11.496253,11.404350
-18.776774,-11.449505,11.342226,5.984717,10.516929,18.435280,11.402758,11.496253,19.523458,18.371585
-18.571073,-11.343003,11.259894,5.801674,10.905420,18.209439,11.281656,11.404350,18.371585,19.132197


[1] 0.18574873 0.07440704 0.28522887 0.47126340 0.45039453 0.44208361 0.46295248
[8] 0.47371523

[1] 0.4309858 0.2727765 0.5340682 0.6864863 0.6711144 0.6648937 0.6804061
[8] 0.6882697

In [47]:
cbind(r_new, r)

r_new,r
0.4309858,0.2404404
0.2727765,0.1257321
0.5340682,0.2950973
0.6864863,0.3803153
0.6711144,0.3791919
0.6648937,0.3397565
0.6804061,0.3410098
0.6882697,0.3940453


In [48]:
cor(r_new, r)

[1] 0.979776

In [50]:
rank(r_new)
rank(r)

[1] 2 1 3 7 5 4 6 8

[1] 2 1 3 7 6 4 5 8